Import all the packages

In [53]:
import pandas as pd
import numpy as np
import re
import stanza
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import nltk

Import dataset

In [54]:
df=pd.read_csv('test.csv')

In [55]:
#df.rename(columns={'Review':'review_text'},inplace=True)

In [56]:
df

,Unnamed: 0,review_id,review_text,review_rating,review_title,sentList
0,1,197033183,I leave this conditioner in my hair for up to ...,4.0,I leave this conditioner in my hair for up to ...,['I leave this conditioner in my hair for up t...
1,2,229195055,OBSESSED!! makes me glow to the gods! So smoot...,5.0,OBSESSED!! makes me glow,"['OBSESSED!!', 'makes me glow to the gods!', '..."
2,3,229199240,I¡¯ve been using this serum for a few days now...,5.0,I love it!,['I¡¯ve been using this serum for a few days n...
3,4,229200919,It was a bit thinner than I anticipated but I ...,3.0,Good,['It was a bit thinner than I anticipated but ...
4,5,229200962,just received my box today and just tried this...,5.0,#ChooseGreenLabs #complimentary @GarnierCan @I...,['just received my box today and just tried th...
5,6,229291864,I received this product for free from topbox c...,5.0,Great quality,['I received this product for free from topbox...
6,7,229509286,I received the Pore Perfection Serum Cream for...,5.0,Amazing,['I received the Pore Perfection Serum Cream f...
7,8,229556279,You can¡¯t go wrong with this product. I love ...,5.0,serum and sunscreen,"['You can¡¯t go wrong with this product.', 'I ..."
8,9,229557206,ok this is a game-changer! I love the way this...,5.0,I LOVE this!,"['ok this is a game-changer!', 'I love the way..."
9,10,229557252,"Great lotion! However, the scent is very stron...",4.0,Great lotion!,"['Great lotion!', 'However, the scent is very ..."


In [31]:
nlp = stanza.Pipeline()

def preprocess(review):
      sentence = review
      sentence = re.sub('[^a-zA-Z]',' ',sentence)
      sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ',sentence)
      sentence = re.sub(r'\s+', ' ',sentence)

      return sentence

2021-11-29 15:28:15 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2021-11-29 15:28:15 INFO: Use device: cpu
2021-11-29 15:28:15 INFO: Loading: tokenize
2021-11-29 15:28:15 INFO: Loading: pos
2021-11-29 15:28:16 INFO: Loading: lemma
2021-11-29 15:28:16 INFO: Loading: depparse
2021-11-29 15:28:16 INFO: Loading: sentiment
2021-11-29 15:28:16 INFO: Loading: constituency
2021-11-29 15:28:16 INFO: Loading: ner
2021-11-29 15:28:17 INFO: Done loading processors!


In [45]:
len(df)

10

In [57]:
aspect = []
for i in range(len(df)):
    txt = df['review_text'][i]
    txt=txt.lower()

    #Create a def to remove all the upper lower case, punctuations, and extra white space
    
    def preprocess(review):
      sentence = review
      sentence = re.sub('[^a-zA-Z]',' ',sentence)
      sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ',sentence)
      sentence = re.sub(r'\s+', ' ',sentence)

      return sentence

    temp=preprocess(txt)
    sentList=nltk.sent_tokenize(temp)

    #Word tokenize and Part-of-speech Tag each word

    for line in sentList:
        txt_list = nltk.word_tokenize(line)
        taggedList = nltk.pos_tag(txt_list)
    print(taggedList)

    sentList

    #Join words with two nouns together(E.g. hair product = hairproduct)then pos tag everything again

    newwordList = []
    flag = 0
    for i in range(0,len(taggedList)-1):
        if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
            newwordList.append(taggedList[i][0]+taggedList[i+1][0])
            flag=1
        else:
            if(flag==1):
                flag=0
                continue
            newwordList.append(taggedList[i][0])
            if(i==len(taggedList)-2):
                newwordList.append(taggedList[i+1][0])
    finaltxt = ' '.join(word for word in newwordList)
    print('final',finaltxt)

    sentList = nltk.sent_tokenize(finaltxt)
    for line in sentList:
        txt_list = nltk.word_tokenize(line)
        taggedList = nltk.pos_tag(txt_list)
    print(taggedList)

    taggedList

    stop_words = set(stopwords.words('english'))
    new_txt_list = nltk.word_tokenize(finaltxt)
    wordsList = [w for w in new_txt_list if not w in stop_words]
    taggedList = nltk.pos_tag(wordsList)

    doc = nlp(finaltxt)
    dep_node = []
    for dep_edge in doc.sentences[0].dependencies:
        dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])
    for i in range(0, len(dep_node)):
        if (int(dep_node[i][1]) != 0):
            dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]
    

    (doc.sentences[0].dependencies)[1]

    featureList = []
    categories = []
    totalfeatureList = []
    print(taggedList)
    for i in taggedList:
        if(i[1]=='JJ' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
            featureList.append(list(i))
            totalfeatureList.append(list(i)) # This list will store all the features for every sentence
            categories.append(i[0])
    #print(featureList)
    #print(totalfeatureList)
    #print(categories)

    fcluster = []
    for i in featureList:
        filist = []
        for j in dep_node:
            if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                if(j[0]==i[0]):
                    filist.append(j[1])
                else:
                    filist.append(j[0])
        fcluster.append([i[0], filist])
    

    finalcluster = []
    dic = {}
    for i in featureList:
        dic[i[0]] = i[1]
    for i in fcluster:
        if(dic[i[0]]=="NN"):
            finalcluster.append(i)
    print(finalcluster)
    aspect.append(finalcluster)
df['cluster']=aspect
    

[('i', 'NN'), ('leave', 'VBP'), ('this', 'DT'), ('conditioner', 'NN'), ('in', 'IN'), ('my', 'PRP$'), ('hair', 'NN'), ('for', 'IN'), ('up', 'IN'), ('to', 'TO'), ('minutes', 'NNS'), ('during', 'IN'), ('each', 'DT'), ('shower', 'NN'), ('leaving', 'VBG'), ('my', 'PRP$'), ('hair', 'NN'), ('soft', 'JJ'), ('and', 'CC'), ('easy', 'JJ'), ('to', 'TO'), ('brush', 'VB'), ('through', 'IN'), ('after', 'IN'), ('the', 'DT'), ('scent', 'NN'), ('never', 'RB'), ('lets', 'VBZ'), ('me', 'PRP'), ('down', 'IN'), ('it', 'PRP'), ('lingers', 'VBZ'), ('all', 'DT'), ('until', 'IN'), ('use', 'VBP'), ('it', 'PRP'), ('again', 'RB'), ('love', 'IN'), ('it', 'PRP')]
final i leave this conditioner in my hair for up to minutes during each shower leaving my hair soft and easy to brush through after the scent never lets me down it lingers all until use it again love it
[('i', 'NN'), ('leave', 'VBP'), ('this', 'DT'), ('conditioner', 'NN'), ('in', 'IN'), ('my', 'PRP$'), ('hair', 'NN'), ('for', 'IN'), ('up', 'IN'), ('to', 'TO

[('bit', 'NN'), ('thinner', 'RB'), ('anticipated', 'VBD'), ('think', 'NN'), ('might', 'MD'), ('sunscreen', 'VB'), ('component', 'JJ'), ('product', 'NN'), ('smells', 'VBZ'), ('amazing', 'JJ'), ('leave', 'VBP'), ('greasy', 'JJ'), ('residue', 'NN'), ('would', 'MD'), ('full', 'JJ'), ('support', 'NN'), ('product', 'NN'), ('garnier', 'NN'), ('although', 'IN'), ('product', 'NN'), ('green', 'JJ'), ('lab', 'JJ'), ('product', 'NN'), ('tested', 'VBN'), ('animals', 'NNS'), ('garnier', 'RBR'), ('products', 'NNS'), ('makes', 'VBZ'), ('hard', 'JJ'), ('support', 'NN'), ('giving', 'VBG'), ('money', 'NN'), ('could', 'MD'), ('put', 'VB'), ('towards', 'NNS'), ('products', 'NNS'), ('tested', 'VBN'), ('animals', 'NNS'), ('wish', 'JJ'), ('products', 'NNS'), ('cruelty', 'VBP'), ('free', 'JJ')]
[]
[('just', 'RB'), ('received', 'VBN'), ('my', 'PRP$'), ('box', 'NN'), ('today', 'NN'), ('and', 'CC'), ('just', 'RB'), ('tried', 'VBN'), ('this', 'DT'), ('omg', 'NN'), ('omg', 'VBZ'), ('rite', 'VB'), ('away', 'RB'), ('

[('received', 'VBN'), ('poreperfection', 'NN'), ('perfectionserum', 'NN'), ('serumcream', 'NN'), ('free', 'JJ'), ('try', 'NN'), ('love', 'NN'), ('texture', 'NN'), ('amazing', 'VBG'), ('smell', 'NN'), ('great', 'JJ'), ('moisture', 'NN'), ('leaves', 'VBZ'), ('skin', 'JJ'), ('amazing', 'VBG'), ('really', 'RB'), ('recommend', 'VB'), ('product', 'NN')]
[]
[('you', 'PRP'), ('can', 'MD'), ('go', 'VB'), ('wrong', 'JJ'), ('with', 'IN'), ('this', 'DT'), ('product', 'NN'), ('love', 'VB'), ('the', 'DT'), ('fact', 'NN'), ('that', 'IN'), ('it', 'PRP'), ('a', 'DT'), ('serum', 'JJ'), ('plus', 'CC'), ('sunscreen', 'JJ'), ('the', 'DT'), ('serum', 'NN'), ('looks', 'VBZ'), ('super', 'JJ'), ('white', 'JJ'), ('when', 'WRB'), ('you', 'PRP'), ('apply', 'VBP'), ('it', 'PRP'), ('but', 'CC'), ('it', 'PRP'), ('absorb', 'RB'), ('and', 'CC'), ('you', 'PRP'), ('will', 'MD'), ('notice', 'VB'), ('the', 'DT'), ('perfect', 'JJ'), ('glow', 'NN'), ('on', 'IN'), ('your', 'PRP$'), ('skin', 'NN')]
final you can go wrong with

In [51]:
df

,Unnamed: 0,review_id,review_text,review_rating,review_title,sentList,cluster
0,1,197033183,I leave this conditioner in my hair for up to ...,4.0,I leave this conditioner in my hair for up to ...,['I leave this conditioner in my hair for up t...,[]
1,2,229195055,OBSESSED!! makes me glow to the gods! So smoot...,5.0,OBSESSED!! makes me glow,"['OBSESSED!!', 'makes me glow to the gods!', '...",[]
2,3,229199240,I¡¯ve been using this serum for a few days now...,5.0,I love it!,['I¡¯ve been using this serum for a few days n...,[]
3,4,229200919,It was a bit thinner than I anticipated but I ...,3.0,Good,['It was a bit thinner than I anticipated but ...,[]
4,5,229200962,just received my box today and just tried this...,5.0,#ChooseGreenLabs #complimentary @GarnierCan @I...,['just received my box today and just tried th...,[]
5,6,229291864,I received this product for free from topbox c...,5.0,Great quality,['I received this product for free from topbox...,[]
6,7,229509286,I received the Pore Perfection Serum Cream for...,5.0,Amazing,['I received the Pore Perfection Serum Cream f...,[]
7,8,229556279,You can¡¯t go wrong with this product. I love ...,5.0,serum and sunscreen,"['You can¡¯t go wrong with this product.', 'I ...",[]
8,9,229557206,ok this is a game-changer! I love the way this...,5.0,I LOVE this!,"['ok this is a game-changer!', 'I love the way...",[]
9,10,229557252,"Great lotion! However, the scent is very stron...",4.0,Great lotion!,"['Great lotion!', 'However, the scent is very ...",[]


In [10]:
df['review_text'][2]

'I¡¯ve been using this serum for a few days now and I can already see a great difference! I use it right after I use the gel wash and it makes my skin feel soft,  moisturized and fresh. I also love how glowing it makes my skin look. One of my favourite things about this serum is that it¡¯s a 3 in 1 formula. I would totally recommend using this!'